In [35]:
from kfp.components import create_component_from_func, InputPath, OutputPath
from typing import Dict, List

%load_ext lab_black

BASE_IMAGE = "quay.io/ibm/kubeflow-notebook-image-ppc64le:latest"


def plot_confusion_matrix(
    input_columns: List[str],
    label_columns: Dict[str, List[str]],
    test_dataset_dir: InputPath(str),
    model_dir: InputPath(str),
    mlpipeline_ui_metadata_path: OutputPath(),
    dataset_split: str = "test",
    num_predictions: int = 0,
    batch_size: int = 20,
    seq_len: int = 7,
):
    """
    Plots a confusion matrix based on the test split of a Huggingface, Numpy or Pandas CSV Dataset and a model trained via Keras or ScikitLearn.

            Parameters:
                    input_columns: Input columns for the model. Examples: ["mel_spectrogram", "pixel_values"].
                    label_columns: Dictionary mapping each label column to a list of possible labels. Example: {"genre": ["Blues", "Rock", "Country"]}
                    test_dataset_dir: Directory where to load test data from. Example: "/blackboard/prep_dataset".
                    model_dir: Directory where to load the model from. Example: "/blackboard/model".
                    dataset_split: Optional name of a dataset's split. Defaults to "test".
                    batch_size: Optional batch size when processing the input dataset. Example: 20.
                    num_predictions: A number restricting how many predictions should be made. Default value 0 indicates that the whole test dataset should be used
                    seq_len: Sequence length for time series datasets. Defaults to 7.
            Returns:
                    mlpipeline_ui_metadata_path: Data to plot a confusion matrix. The plotted confusion matrix can be viewed via Kubeflow UI's Vizualization for this component inside a pipeline run.
    """
    from collections.abc import Iterable
    import json
    import logging
    import numpy as np
    import pandas as pd
    import sys
    import tensorflow as tf
    from tensorflow import keras
    import os
    import joblib
    import lightgbm

    logging.basicConfig(
        stream=sys.stdout,
        level=logging.INFO,
        format="%(levelname)s %(asctime)s: %(message)s",
    )
    logger = logging.getLogger()

    def relabel_map(item, label):
        label = [label]
        return item, label

    files = os.listdir(test_dataset_dir)
    csv = list(filter(lambda x: ".csv" in x, files))
    npz = list(filter(lambda x: ".npz" in x, files))

    # pandas dataset
    if len(csv) > 0:
        path = os.path.join(test_dataset_dir, csv[0])
        lc = list(label_columns.keys())
        test_dataset = tf.data.experimental.make_csv_dataset(
            path,
            batch_size=1,
            label_name=lc[0],
            select_columns=input_columns + lc,
        )
        test_dataset = test_dataset.map(relabel_map)
        test_dataset = test_dataset.batch(batch_size)
        logger.info("Loaded CSV dataset.")
    # numpy dataset
    elif len(npz) > 0:
        data = np.load(os.path.join(test_dataset_dir, npz[0]))
        x, y = data["x"], data["y"]
        test_dataset = keras.preprocessing.timeseries_dataset_from_array(
            x, y, sequence_length=seq_len, batch_size=batch_size
        )
        logger.info("Loaded numpy timeseries dataset.")
    # plain tensorflow dataset
    else:
        test_dataset = tf.data.experimental.load(test_dataset_dir)
        logger.info("Loaded TF dataset.")

    if num_predictions == 0:
        try:
            num_predictions = test_dataset.__len__()
        except Exception as e:
            logger.warning(e)
            num_predictions = test_dataset.cardinality().numpy()

    tff, pkl = [], []
    if os.path.isdir(model_dir):
        models = os.listdir(model_dir)
        tff = list(filter(lambda x: ".pb" in x, models))
        pkl = list(filter(lambda x: ".pkl" in x, models))

    # keras/tf model
    if len(tff) > 0:
        model = keras.models.load_model(model_dir)
    # sklearn model
    elif len(pkl) > 0:
        model = joblib.load(os.path.join(model_dir, pkl[0]))
    # lightgbm/sklearn/otherwise pickled model
    else:
        try:
            model = joblib.load(model_dir)
        except Exception as e:
            raise Exception("Model format not yet supported:", e)

    # see: https://github.com/huggingface/datasets/issues/4772
    if "labels" in label_columns:
        label_columns["label"] = label_columns.pop("labels")

    def ensure_encoding(label_tensor):
        rank = label_tensor.shape.rank

        if rank != 1 and rank != 2:
            err = f"Rank of label tensor has to be 1 or 2 but found rank {rank}!"
            raise Exception(err)

        # transform one-hot vector into integer
        return tf.math.argmax(label_tensor, axis=rank - 1)

    def prediction_to_encoded_tensor(prediction):
        result = []
        if isinstance(prediction, np.ndarray):
            if len(prediction) == 1:
                pred = prediction[0]
                if isinstance(pred, np.ndarray):
                    if len(pred) == 1:
                        result = round(pred[0])
                    elif len(pred) > 1:
                        result.append(np.argmax(pred, axis=0))
                    else:
                        err = f"Unsupport prediction array size: {len(pred[0])}"
                        raise Exception(err)
                elif isinstance(pred, (np.floating, float)):
                    result.append(round(pred))
                elif isinstance(pred, (np.integer, int)):
                    result.append(pred)
                else:
                    err = f"Unsupport prediction type: {type(pred)}"
                    raise Exception(err)
            elif len(prediction) > 1:
                result = np.argmax(prediction, axis=0)
            else:
                err = f"Unsupport prediction length: {len(prediction)}"
                raise Exception(err)
        elif isinstance(prediction, tf.Tensor):
            result = ensure_encoding(prediction)
        elif isinstance(prediction, (np.floating, float)):
            result = round(prediction)
        elif isinstance(prediction, (np.integer, int)):
            result = prediction
        else:
            err = f"Unsupport model prediction type: {type(prediction)}"
            raise Exception(err)

        if isinstance(result, tf.Tensor):
            if len(result.shape) == 0:
                result = [result]
        elif isinstance(result, dict) or isinstance(result, list):
            if len(result) == 0:
                result = [result]
        elif isinstance(result, (np.integer, int)):
            result = [result]
        else:
            err = f"Unsupport result type: {len(result)}"
            raise Exception(err)

        return result

    def merge_to_dict(dictionary, key, tensor):
        if key in dictionary:
            dictionary[key] = tf.concat([dictionary.pop(key), tensor], axis=0)
        else:
            dictionary[key] = tensor

    def process_tensors(dictionary, tensors, label_columns, index2label):
        if isinstance(tensors, dict):
            # Multi-label support
            for key, tensor in tensors.items():
                encoded_tensor = prediction_to_encoded_tensor(tensor)
                merge_to_dict(dictionary, key, encoded_tensor)
        elif isinstance(tensors, list) or isinstance(tensors, np.ndarray):
            # Multi-label support; assume order & get name from dataset
            for idx, tensor in enumerate(tensors):
                encoded_tensor = prediction_to_encoded_tensor(tensor)
                if len(index2label) == 1:
                    label = index2label[0]
                else:
                    label = index2label[idx]
                merge_to_dict(dictionary, label, encoded_tensor)
        elif isinstance(tensors, tf.Tensor):
            # Assuming single label
            if len(label_columns) > 1:
                err = f"Model provides only 1 output but got {len(label_columns)} label columns!"
                raise Exception(err)
            key = next(iter(label_columns))

            for tensor in tensors:
                encoded_tensor = prediction_to_encoded_tensor(tensor)
                merge_to_dict(dictionary, key, encoded_tensor)
        else:
            err = f"Unsupported tensors type: {type(tensors)}!"
            raise Exception(err)

    # Get label indexes & names from dataset

    index2label = dict()
    label_specs = test_dataset.element_spec[1]
    if isinstance(label_specs, Iterable):
        for i, label in enumerate(label_specs):
            index2label[i] = label
    else:
        # Fall-back to Huggingface dataset
        for i, label in enumerate(label_columns):
            index2label[i] = label
    logging.info(f"Indexes mapped to labels: {index2label}")

    # each dataset_batch is of size batch_size; results are aggregate inside this loop
    y_true = dict()
    y_pred = dict()
    for dataset_batch in test_dataset.take(num_predictions):
        label_tensors = dataset_batch[1]
        process_tensors(y_true, label_tensors, label_columns, index2label)

        feature_tensors = dataset_batch[0]
        try:
            # keras/tf model
            predictions = model.predict(feature_tensors)
        except ValueError:
            # lightgbm/sklearn model trained with numpy arrays
            feature_tensors = np.array(list(feature_tensors.values())).T[
                0
            ]  # .reshape(1, -1)
            predictions = model.predict(feature_tensors)
        process_tensors(y_pred, predictions, label_columns, index2label)

    logging.info(f"Final labels: {y_true}")
    logging.info(f"Final predictions: {y_pred}")

    confusion_matrices = []
    for label_column, labels in label_columns.items():
        confusion_matrix = tf.math.confusion_matrix(
            labels=y_true[label_column],
            predictions=y_pred[label_column],
            num_classes=len(labels),
        )

        data = []
        for target_index, target_row in enumerate(confusion_matrix):
            for predicted_index, count in enumerate(target_row):
                data.append(
                    (labels[target_index], labels[predicted_index], count.numpy())
                )

        df = pd.DataFrame(data, columns=["target", "predicted", "count"])

        confusion_matrices.append(
            {
                "type": "confusion_matrix",
                "format": "csv",
                "schema": [
                    {"name": "target", "type": "CATEGORY"},
                    {"name": "predicted", "type": "CATEGORY"},
                    {"name": "count", "type": "NUMBER"},
                ],
                "storage": "inline",
                "source": df.to_csv(
                    columns=["target", "predicted", "count"], header=False, index=False
                ),
                "labels": labels,
            }
        )

    metadata = {"outputs": confusion_matrices}

    logger.info("Dumping mlpipeline_ui_metadata...")
    with open(mlpipeline_ui_metadata_path, "w") as metadata_file:
        json.dump(metadata, metadata_file)

    logger.info("Finished.")


load_dataset_comp = create_component_from_func(
    func=plot_confusion_matrix,
    output_component_file="component.yaml",
    base_image=BASE_IMAGE,
    packages_to_install=["lightgbm"],
)

The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black
